# Análisis de Pose en Actividades Deportivas
Se deben instalar las librerías antes de pasar a usar el modelo.

# Importación de Datos e Instalación de Librerías

In [ ]:
pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 8.6 MB/s eta 0:00:00


In [ ]:
pip install timm torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.9 MB/s eta 0:00:00


In [ ]:
import cv2
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/107.7 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Análisis y Organización del Dataset


In [ ]:
# Función para convertir videos en frames con poses aisladas en fondo negro
from PIL import Image
from ultralytics import YOLO
import numpy as np

 # La función recibe 'name' como nombre para el archivo, 'path' como ubicación del archivo y 'end' como la ubicación donde se quiere enviar.
def crear_dataset(name, path, end):
  model = YOLO('yolov8n-pose.pt') # Modelo YOLO Pose para la detección
  results = model(path) # Se aplica detección en el video
  n=0
  j=0
  while j<len(results):
    n = n+1
    original = results[j].orig_shape
    h = int(original[0])
    w = int(original[1])
    black_img = np.zeros((h,w,3),dtype=np.uint8) # Se genera imagen negra con alto y largo de la imagen original
    img_array = results[j].plot(img=black_img, labels=False, boxes=False, probs=False)
    im = Image.fromarray(img_array[..., ::-1])
    # im.show() en caso de querer ver las imágenes
    im_path = end +'/results_'+str(name)+'_'+str(n)+'.jpg'
    im.save(im_path)
    j = j+1 # Frame a frame (se puede hacer saltando frames si son muchos videos)

In [ ]:
import os
import shutil
def copiar_dataset(path,end):
  files = os.listdir(path)
  for file in files[:200]:
    start = os.path.join(path, file)
    destiny = os.path.join(end, file)
    shutil.copy(start, destiny)

copiar_dataset('/content/drive/MyDrive/Base de Datos Duckietown/Dataset/Test/Standing', '/content/drive/MyDrive/Base de Datos Duckietown/Dataset2/Test/Standing')


In [ ]:
import os
import shutil
files = os.listdir('/content/drive/MyDrive/Base de Datos Duckietown/Dataset2/Test/Standing')
print(len(files))

200


#Entrenamiento

In [ ]:
import torch
import timm
import torch.optim as optim

# Se registra la cantidad de ejercicios distintos, en este caso 5
num_classes = 4

# Se parte de un modelo preentrenado, en este caso una mobilenet
model = timm.create_model('timm/mobilenetv3_large_100.ra_in1k', pretrained=True, num_classes=num_classes)

# Herramientas para la clasificación
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.safetensors:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

In [ ]:
import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

cuda = torch.cuda.is_available()
#Importación datasets
train_data_path = '/content/drive/MyDrive/Base de Datos Duckietown/Dataset2/Train'
val_data_path = '/content/drive/MyDrive/Base de Datos Duckietown/Dataset2/Test'

#Transformaciones a las imágenes (revisar Resize!)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

#ImageFolders para los datasets (pendiente agregar transforms)
train_dataset = ImageFolder(root=train_data_path, transform=transform)
val_dataset = ImageFolder(root=val_data_path, transform=transform)

print(train_dataset.classes)
print(val_dataset.classes)

#Definir dataset_loaders
batch_size = 128
kwargs = {'num_workers': 2, 'pin_memory': True} if cuda else {}
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, **kwargs)

#Número de épocas
num_epochs = 10

if cuda:
    model.cuda()
#Entrenamiento del modelo con set Train
for epoch in range(num_epochs):
    total_loss = 0
    val_total_loss = 0
    for inputs, labels in train_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()

#Validación del modelo con set Test
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs = val_inputs.cuda()
            val_labels = val_labels.cuda()
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_labels)
            val_total_loss += val_loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Val Loss: {val_total_loss/len(val_loader)}')

#Guardado del modelo
model_path = '/content/drive/MyDrive/Base de Datos Duckietown/Modelo2/duckietown-pose-model.pth'
torch.save(model.state_dict(), model_path)


['PesoMuerto', 'Press', 'Squat', 'Standing']
['PesoMuerto', 'Press', 'Squat', 'Standing']
Epoch 1/10, Loss: 1.5387806353074582, Val Loss: 2.8984149737017497
Epoch 2/10, Loss: 0.0355580032390373, Val Loss: 3.759730134691511
Epoch 3/10, Loss: 0.03388885615541684, Val Loss: 3.2309413637433733
Epoch 4/10, Loss: 0.023464128726945175, Val Loss: 3.800873041152954
Epoch 5/10, Loss: 0.021926752442926623, Val Loss: 3.5556141308375766
Epoch 6/10, Loss: 0.0069196115979883125, Val Loss: 3.8453985324927737
Epoch 7/10, Loss: 0.006048753055603645, Val Loss: 4.067856533186776
Epoch 8/10, Loss: 0.0030703890327004046, Val Loss: 4.484972340720041
Epoch 9/10, Loss: 0.006144691128998726, Val Loss: 4.652389568941934
Epoch 10/10, Loss: 0.005202292979767911, Val Loss: 4.905182523386819


# Uso del Modelo para la detección de poses

In [ ]:
# Se definen las herramientas a usar para el programa final
from ultralytics import YOLO
import numpy as np
import torch
import timm
from torchvision import transforms
from PIL import Image

# Carga del modelo
model_pose = YOLO('yolov8n-pose.pt')
model_path = '/content/drive/MyDrive/Base de Datos Duckietown/Modelo2/duckietown-pose-model.pth'
num_classes = 4
model = timm.create_model('timm/mobilenetv3_large_100.ra_in1k', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(model_path)) # Se agregan los pesos del entrenamiento
model.eval() # Se deja el modelo en modo evaluación

# Función para aislar en un fondo negro la pose de una imagen.
def isolate_pose(path):
  model = YOLO('yolov8n-pose.pt') # Modelo YOLO Pose para la detección
  results = model(path)
  original = results[0].orig_shape
  h = int(original[0])
  w = int(original[1])
  black_img = np.zeros((h,w,3),dtype=np.uint8) # Se genera imagen negra con alto y largo de la imagen original
  img_array = results[0].plot(img=black_img, labels=False, boxes=False, probs=False)
  im = Image.fromarray(img_array[..., ::-1])
  return im

# Transformaciones a realizar en las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Preprocesamiento imagen
def preprocess_image(image):
    #image = Image.open(image)
    image = transform(image)
    image = image.unsqueeze(0)
    return image

# Función para predecir con el modelo
def model_predict(model, image):
    with torch.no_grad():
        outputs = model(image)
    return outputs

RuntimeError: ignored

Esta última corresponde a la función la que se realiza el análisis de pose, sólo se debe correr el código de arriba para que pueda funcionar y usar la función dándole como única variable la ruta hacia la imagen/video que se quiere estudiar.

In [ ]:
# Función para detectar el ejercicio que se está realizando a partir de una imagen o video
# La base de datos del entrenamiento actual contiene: persona de pie, sentadilla, flexión, peso muerto y press militar.

def pose_detect(image_path):
  pose_img = isolate_pose(image_path)
  pre_img = preprocess_image(pose_img)
  results = model_predict(model, pre_img)
  probabilidades, predicciones = torch.max(results, 1)
  print(f"Clase predicha: {predicciones.item()}, Probabilidad: {probabilidades.item()}")
  detected_pose = predicciones.item()
  return detected_pose


#pose_detect('/content/drive/MyDrive/Base de Datos Duckietown/PoseFlex.png')
pose_detect('/content/drive/MyDrive/Base de Datos Duckietown/PressTest.png')

NameError: ignored